### A Demonstration of Training and Evaluating A Toxicity Classifier
Thomas Zeng
9/20/22

## Notebook setup

Autoreload extension is used so that everyime you modify python files imported by notebook, you don't have to restart the notebook

In [1]:
%load_ext autoreload
%autoreload 2 # 2 is mode where every file you import gets reloaded when you run a block

In [4]:
import torch
from utils import load_jigsaw
from dataloaders import InfiniteDataLoader
from models import BertClassifier
from train_eval import train, evaluate

Pytorch can do it's computation in three modes.

1. Cuda: if you have a Nvidia gpu and you have a version of PyTorch installed that has cuda enabled (fastest speed)
2. Metal: if you have an Apple device with Apple Silicon (M1 or M2) -- this implementation of PyTorch is new, not feature complete and rather buggy (medium speed)
3. CPU: this is just the cpu (slowest speed)

In [5]:
if torch.cuda.is_available():
    print('Using GPU')
    DEVICE = torch.device('cuda')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    # macbooks can use metal if the right version of pytorch is installed
    print('Using Metal')
    DEVICE = torch.device('mps')
else:
    print('Using cpu')
    DEVICE = torch.device('cpu')

Using Metal


## Data Initialization

Pytorch requires its datasets to be ascessible following the [datasets api](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files).


In [29]:
train_data, test_data = load_jigsaw(device=DEVICE, demo_mode=True) #demo_mode=True only loads a subset of entire dataset so as to make training faster for demonstration purposes

100%|██████████| 1000/1000 [00:00<00:00, 1519.90it/s]


PyTorch models receive data for training and inference through a dataloader. A dataloader samples from a dataset and returns a batch of samples each time it is called.

In [53]:
train_loader =  InfiniteDataLoader(train_data, batch_size=16)
test_loader = InfiniteDataLoader(test_data, batch_size=32)

## Model and Training Stuff Initialization

For our model, we use transfer learning, i.e. we use a pre-trained model -- DistilBert -- implemented by HuggingFace. Our `BertClassifier` is a simple wrapper arround their model.

In [50]:
model = BertClassifier(device=DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

An epoch is the number of times you go through your datase during training. That is you have trained for 1 epoch when you have seen every sample in your training dataset once.<br>
The loss function is the training objective we want our model to minimize.<br>
The optimizer is used at every time step i.e. everyime we compute the loss and its gradient. It is used to update the model weights.

In [51]:
epochs = 5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

## Train and Evaluation

We train for 5 epochs

In [56]:
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train(train_loader, model, loss_fn, optimizer, verbose=True, use_tqdm=True)

Epoch 1/5


  2%|▏         | 14/625 [00:36<26:28,  2.60s/it] 


KeyboardInterrupt: 

We evaluate our reults on the test data.

In [54]:
evaluate(test_loader, model, get_loss=True, verbose=True)

100%|██████████| 32/32 [00:17<00:00,  1.84it/s]

Loss: 0.69342440366745
Accuracy: 0.497, Sensitivity: 0.994, Specificity: 0.0


(tensor(0.6934), 0.497, 0.994, 0.0)

Our results are pretty bad, which suggests we need hyperparameter tuning.